### RQ1. Does support for specific policies (carbon taxes or emissions standards) shift during elections?
#### Global Panel VAR(1) System (Support for Carbon Taxes)
With 11 time-varying variables; cc4_world , cc4_wealthUS, cc4_poorUS, cc4_comm, cc4_famheal, cc4_famecon, ccSolve, cc_pol_tax, cc_pol_car, pol_party, pol_ideology, we are estimating the following system of 11 equations. Each variable at time t is regressed on the **lagged values of all 11 variables** at time t - 1:

\begin{cases}
\text{cc\_pol\_tax}_t = \alpha_1 + \sum_{j=1}^{11} \beta_{1j} \cdot X_{j,t-1} + \epsilon_{1t} \\
\text{cc\_pol\_car}_t = \alpha_2 + \sum_{j=1}^{11} \beta_{2j} \cdot X_{j,t-1} + \epsilon_{2t} \\
\text{ccSolve}_t = \alpha_3 + \sum_{j=1}^{11} \beta_{3j} \cdot X_{j,t-1} + \epsilon_{3t} \\
\text{cc4\_world}_t = \alpha_4 + \sum_{j=1}^{11} \beta_{4j} \cdot X_{j,t-1} + \epsilon_{4t} \\
\text{cc4\_wealthUS}_t = \alpha_5 + \sum_{j=1}^{11} \beta_{5j} \cdot X_{j,t-1} + \epsilon_{5t} \\
\text{cc4\_poorUS}_t = \alpha_6 + \sum_{j=1}^{11} \beta_{6j} \cdot X_{j,t-1} + \epsilon_{6t} \\
\text{cc4\_comm}_t = \alpha_7 + \sum_{j=1}^{11} \beta_{7j} \cdot X_{j,t-1} + \epsilon_{7t} \\
\text{cc4\_famheal}_t = \alpha_8 + \sum_{j=1}^{11} \beta_{8j} \cdot X_{j,t-1} + \epsilon_{8t} \\
\text{cc4\_famecon}_t = \alpha_9 + \sum_{j=1}^{11} \beta_{9j} \cdot X_{j,t-1} + \epsilon_{9t} \\
\text{pol\_party}_t = \alpha_{10} + \sum_{j=1}^{11} \beta_{10j} \cdot X_{j,t-1} + \epsilon_{10t} \\
\text{pol\_ideology}_t = \alpha_{11} + \sum_{j=1}^{11} \beta_{11j} \cdot X_{j,t-1} + \epsilon_{11t}
\end{cases}

Where:

- $ X_{j,t-1} $ denotes the value of the $ j $-th variable in the system at time $ t-1 $
- $ \alpha_i $ is the intercept for equation $ i $
- $ \beta_{ij} $ is the coefficient for the effect of variable $ j $ on outcome $ i $
- $ \epsilon_{it} $ is the idiosyncratic error term for individual $ i $ at time $ t $

#### Python Code for Global Panel VAR(1) – RQ1

In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load the dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Define the full set of 11 time-varying variables
variables = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS", "cc4_comm",
    "cc4_famheal", "cc4_famecon", "ccSolve",
    "cc_pol_tax", "cc_pol_car", "pol_party", "pol_ideology"
]

# Sort and create lagged versions of all variables
data = data.sort_values(by=['PID', 'wave'])
for var in variables:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop rows with any missing values in the lagged variables
lagged_vars = [f'{v}_lag' for v in variables]
panel_var_data = data.dropna(subset=lagged_vars)

# Estimate the equation of interest: cc_pol_tax_t as outcome
X = panel_var_data[lagged_vars]
y = panel_var_data['cc_pol_tax']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results for cc_pol_tax_t equation
print("\nEquation: cc_pol_tax_t (VAR(1)) in the global system")
print(model.summary())


Equation: cc_pol_tax_t (VAR(1)) in the global system
                            OLS Regression Results                            
Dep. Variable:             cc_pol_tax   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     192.0
Date:                Mon, 14 Apr 2025   Prob (F-statistic):          7.98e-223
Time:                        22:04:02   Log-Likelihood:                -2297.9
No. Observations:                1722   AIC:                             4620.
Df Residuals:                    1710   BIC:                             4685.
Df Model:                          11                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

#### Panel VAR Analysis – RQ1 (Support for Carbon Taxes)
The dependent variable in this equation is `cc_pol_tax_t` (support for carbon taxes at time $ t $), and the rest of the variables are predictors. All variables are treated as jointly endogenous. The model was estimated using pooled OLS with standard errors clustered by respondent (`PID`), based on 1,722 complete observations.

#### Results: Equation for `cc_pol_tax_t`
| Predictor          | Coefficient | Significance    | Interpretation                                                             |
|--------------------|-------------|-----------------|----------------------------------------------------------------------------|
| `cc4_world_lag`    | **0.188**   | *** p < 0.01    | Global harm perception increases carbon tax support                        |
| `cc4_wealthUS_lag` | –0.051      | * p < 0.05      | Concern for wealthy communities slightly decreases support                 |
| `cc4_poorUS_lag`   | **0.061**   | * p < 0.05      | Concern for poor communities increases support                             |
| `cc4_comm_lag`     | –0.027      | Not significant | No effect from local harm perception                                       |
| `cc4_famheal_lag`  | –0.004      | Not significant | No effect from family health concern                                       |
| `cc4_famecon_lag`  | +0.025      | Not significant | No significant economic concern effect                                     |
| `ccSolve_lag`      | **0.093**   | *** p < 0.01    | Past willingness to pay significantly predicts carbon tax support          |
| `cc_pol_tax_lag`   | **0.352**   | *** p < 0.01    | Strong autoregressive effect; carbon tax support is stable over time       |
| `cc_pol_car_lag`   | **0.138**   | *** p < 0.01    | Past support for emissions standards boosts current support for carbon tax |
| `pol_party_lag`    | **0.102**   | *** p < 0.01    | More left-leaning party affiliation predicts increased support             |
| `pol_ideology_lag` | +0.048      | Not significant | Political ideology has a positive but non-significant effect               |

Model fit: $ R^2 = 0.498 $

This global Panel VAR model provides strong evidence that **support for carbon taxes is shaped by a wide array of interrelated factors**, particularly:
- **Perceptions of global and poor-community climate harm** have significant positive effects on support, while concern for wealthy communities slightly suppresses support.
- **Willingness to pay** significantly predicts increased support for carbon taxation, indicating that financial climate commitment co-evolves with policy preferences.
- **Support for regulatory policies (emissions standards)** at $ t-1 $ positively influences carbon tax support, suggesting alignment across policy types.
- **Political party identification** strongly predicts changes in support, while ideology contributes positively but is not statistically significant when controlling for other attitudes.

> Support for climate policies, like carbon taxes, is stable but changes over time. The global Panel VAR(1) model shows that support is influenced by earlier support for related policies, willingness to pay, and concern about climate harm. Although the model does not directly measure election effects, the results suggest that changes in support are part of broader shifts in climate attitudes, not just reactions to election periods.

> Our study includes both carbon taxes (`cc_pol_tax`) and emissions standards (`cc_pol_car`) because these are the two climate policy variables measured across all waves of the survey. Since no single policy was designated as the primary focus of the analysis, both are examined in parallel to assess how support for climate policies evolves over time and how it is shaped by related attitudes such as perceived harm, willingness to pay, and political identity. This approach ensures that findings are not limited to one specific policy type, but reflect broader patterns in climate policy support. Below we will explore the support for emissions standards

In [5]:
import pandas as pd
import statsmodels.api as sm

# Load your data
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Define variables
variables = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS", "cc4_comm",
    "cc4_famheal", "cc4_famecon", "ccSolve",
    "cc_pol_tax", "cc_pol_car", "pol_party", "pol_ideology"
]

# Sort and create lags
data = data.sort_values(by=['PID', 'wave'])
for var in variables:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop missing
lagged_vars = [f'{v}_lag' for v in variables]
panel_var_data = data.dropna(subset=lagged_vars)

# Estimate equation: cc_pol_car_t
X = panel_var_data[lagged_vars]
y = panel_var_data['cc_pol_car']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Output
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             cc_pol_car   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     148.1
Date:                Mon, 14 Apr 2025   Prob (F-statistic):          6.54e-190
Time:                        22:04:14   Log-Likelihood:                -2187.4
No. Observations:                1722   AIC:                             4399.
Df Residuals:                    1710   BIC:                             4464.
Df Model:                          11                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6845      0.092  

### Panel VAR Analysis – RQ1 (Support for Emissions Standards)
#### Results: Equation for `cc_pol_car_t`
| Predictor               | Coefficient | Significance | Interpretation |
|-------------------------|-------------|--------------|----------------|
| `cc_pol_car_lag`        | **0.429**   | *** p < 0.01 | Support is highly stable over time |
| `cc_pol_tax_lag`        | **0.107**   | *** p < 0.01 | Support for carbon taxes boosts support for emissions standards |
| `cc4_world_lag`         | **0.152**   | *** p < 0.01 | Perceived global harm increases support |
| `cc4_poorUS_lag`        | **0.098**   | *** p < 0.01 | Concern for poor communities increases support |
| `cc4_wealthUS_lag`      | –0.052      | ** p < 0.05  | Concern for wealthy communities decreases support |
| `cc4_comm_lag`          | 0.004       | Not significant | No effect from local community concern |
| `cc4_famheal_lag`       | 0.000       | Not significant | No effect from personal health concern |
| `cc4_famecon_lag`       | –0.020      | Not significant | No effect from economic concern |
| `ccSolve_lag`           | 0.017       | Not significant | WTP does not significantly predict policy support here |
| `pol_party_lag`         | **0.037**   | ** p < 0.05 | Leftward shifts in party identification increase support |
| `pol_ideology_lag`      | **0.060**   | ** p < 0.05 | More liberal ideology predicts greater support |

Model fit: \( R^2 = 0.506 \)

Support for emissions standards is stable over time and influenced by several key attitudes. Like carbon tax support, it is positively shaped by prior concern about **global climate harm** and **inequity (harm to poor communities)**, while concern for **wealthy communities** is associated with reduced support. 

Interestingly, unlike carbon tax support, **willingness to pay does not significantly predict support for emissions standards**, suggesting that **financial commitment** is more closely tied to **market-based policies** (e.g., carbon pricing), while regulatory support may be driven more by **equity and ideological considerations**.

Both **political party identification** and **ideology** significantly predict changes in support, reinforcing the idea that emissions regulation is a **politically influenced** issue. 

> So, the anwer to RQ1 is **Yes**.

---

### RQ2. How does willingness to pay for climate solutions vary during elections, and what factors influence it? ~~Are voters more likely to prioritize low-cost climate solutions during election periods?~~

This question has two parts:
1. **How does WTP evolve across time (especially around elections)?**
2. **What factors (e.g., harm perception, policy support) influence WTP, and vice versa?**

**What We Can Do with Panel VAR**
| Question                                                                                         | Panel VAR                                           |
|--------------------------------------------------------------------------------------------------|-----------------------------------------------------|
| Is WTP influenced by its **own past values**?                                                    | Yes                                                 |
| Is WTP influenced by **past values of other variables** (e.g., harm perception, policy support)? | Yes                                                 |
| Does WTP **predict** support for climate policies or harm perception?                            | Yes                                                 |
| Does the **system of attitudes shift dynamically** over time?                                    | Yes                                                 |

#### Python Code for Global Panel VAR(1) – RQ2

In [7]:
import pandas as pd
import statsmodels.api as sm

# Load and prep data
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

variables = ["cc4_world", "cc4_wealthUS", "cc4_poorUS", "cc4_comm", "cc4_famheal", "cc4_famecon", "ccSolve", "cc_pol_tax", "cc_pol_car", "pol_party", "pol_ideology"]

# Create lags
data = data.sort_values(by=['PID', 'wave'])
for var in variables:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop missing
lagged_vars = [f'{v}_lag' for v in variables]
panel_var_data = data.dropna(subset=lagged_vars)

# Estimate equation: ccSolve_t
X = panel_var_data[lagged_vars]
y = panel_var_data['ccSolve']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Output
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     11.66
Date:                Mon, 14 Apr 2025   Prob (F-statistic):           1.65e-20
Time:                        23:04:16   Log-Likelihood:                -2353.4
No. Observations:                1722   AIC:                             4731.
Df Residuals:                    1710   BIC:                             4796.
Df Model:                          11                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.9160      0.087  

### Panel VAR Analysis – RQ2 (Willingness to Pay as Outcome)
To assess how willingness to pay for climate solutions evolves over time and what factors influence it, I examined the `ccSolve_t` equation within a global Panel VAR(1) system. This model includes 11 time-varying attitudinal and political variables, each treated as endogenous and lagged by one wave. The model was estimated using pooled OLS with cluster-robust standard errors at the individual level, based on 1,722 complete observations across three survey waves.

| Predictor          | Coefficient | Significance    | Interpretation                                       |
|--------------------|-------------|-----------------|------------------------------------------------------|
| `ccSolve_lag`      | 0.054       | * p = 0.052     | WTP is modestly self-reinforcing over time           |
| `cc_pol_tax_lag`   | **0.116**   | *** p < 0.001   | Support for carbon taxes increases future WTP        |
| `cc4_world_lag`    | 0.064       | † p = 0.076     | Global concern may positively influence WTP          |
| `cc4_comm_lag`     | –0.057      | Not significant | No clear effect from local harm perception           |
| `cc4_famheal_lag`  | 0.039       | Not significant | No effect from family health concern                 |
| `cc4_poorUS_lag`   | 0.030       | Not significant | No effect from concern for poor communities          |
| `cc4_wealthUS_lag` | 0.001       | Not significant | No effect from concern for wealthy communities       |
| `cc4_famecon_lag`  | 0.038       | Not significant | No effect from economic concern                      |
| `cc_pol_car_lag`   | –0.035      | Not significant | Support for emissions standards does not predict WTP |
| `pol_party_lag`    | –0.012      | Not significant | No direct effect from party affiliation              |
| `pol_ideology_lag` | 0.057       | † p = 0.094     | Political ideology may predict slightly higher WTP   |

Model fit: \( R^2 = 0.069 \)

Willingness to pay for climate solutions is **weakly stable** over time and appears to be shaped primarily by **prior support for carbon taxes**. Individuals who supported carbon pricing in the previous wave were significantly more likely to express willingness to pay in the next. There is also some evidence that global climate concern and political ideology may be associated with increases in WTP, though these effects are marginally significant.

Other harm perception variables, including concern for local or personal impacts, do not significantly influence WTP in this model. Likewise, support for regulatory policies (`cc_pol_car`) and political party identification do not predict willingness to pay when controlling for all other variables.

These findings suggest that **willingness to pay is primarily shaped by prior policy alignment**, and that shifts in WTP during elections or other periods are likely driven by broader attitudinal shifts rather than direct political or emotional responses.

> So, the anwer to RQ2 is **No**.

---

### RQ3. How do perceptions of (personal and community-level) climate change impacts influence policy support during elections?
> **Original research question:**  
How do perceptions of (personal and community-level) climate change impacts influence policy support during elections?

> **Panel VAR framing:**  
Are **perceived harm** and **policy support** **dynamically interrelated** across time?  
Does **past harm perception** predict **current policy support**, and vice versa?

### Suggested Variables for the Panel VAR System
To capture the core logic of RQ3:
| Variable      | Description                                                          |
|---------------|----------------------------------------------------------------------|
| `cc4_comm`    | Perceived harm to local communities                                  |
| `cc4_famheal` | Perceived harm to family health (optional: for personal-level angle) |
| `cc_pol_tax`  | Support for carbon taxes (policy support variable of focus)          |

This model lets us test:
- Whether **prior harm perceptions** (communal or personal) predict changes in **policy support**
- Whether **policy support itself** influences **future harm perceptions**, possibly via motivated reasoning
- The **bidirectional dynamics** between these constructs over time

In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and create lagged variables
data = data.sort_values(by=['PID', 'wave'])
data['cc4_comm_lag'] = data.groupby('PID')['cc4_comm'].shift(1)
data['cc4_famheal_lag'] = data.groupby('PID')['cc4_famheal'].shift(1)
data['cc_pol_tax_lag'] = data.groupby('PID')['cc_pol_tax'].shift(1)

# Drop rows with missing values
panel_var_data = data.dropna(subset=['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag'])

# Equation 1: policy support as outcome
X1 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y1 = panel_var_data['cc_pol_tax']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 2: community-level harm as outcome
X2 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y2 = panel_var_data['cc4_comm']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 3: family-level harm as outcome
X3 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_tax_lag']]
y3 = panel_var_data['cc4_famheal']
X3 = sm.add_constant(X3)
model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results
print(summary_col([model1, model2, model3],
                  stars=True,
                  model_names=['cc_pol_tax_t', 'cc4_comm_t', 'cc4_famheal_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


                cc_pol_tax_t cc4_comm_t cc4_famheal_t
-----------------------------------------------------
const           0.7809***    0.6603***  0.4720***    
                (0.0719)     (0.0611)   (0.0588)     
cc4_comm_lag    0.1402***    0.4848***  0.3034***    
                (0.0327)     (0.0323)   (0.0339)     
cc4_famheal_lag 0.0478       0.1427***  0.3372***    
                (0.0296)     (0.0293)   (0.0350)     
cc_pol_tax_lag  0.5511***    0.1484***  0.1244***    
                (0.0251)     (0.0199)   (0.0202)     
R-squared       0.4260       0.4816     0.4543       
R-squared Adj.  0.4250       0.4807     0.4533       
N               1722         1722       1722         
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ3 (Carbon Tax as Outcome)
To explore the dynamic relationship between perceived climate change harm and support for climate policy, I estimated a Panel VAR(1) model including three variables:
- `cc_pol_tax`: support for a carbon tax  
- `cc4_comm`: perceived harm from climate change to one's local community  
- `cc4_famheal`: perceived harm to one’s family’s health

Each variable was regressed on its own lag and the lagged values of the other two variables. The model was estimated using pooled OLS with clustered standard errors by individual (PID), covering three waves and 1,722 complete observations.

### Results
**Equation 1: Support for Carbon Tax (`cc_pol_tax_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.140** | *** p < 0.01 |
| `cc4_famheal_lag` | 0.048 | Not significant |
| `cc_pol_tax_lag` | **0.551** | *** p < 0.01 |

→ Support for carbon taxes is strongly self-reinforcing over time. Importantly, perceptions of **community-level harm** significantly increase future support for carbon taxes, while **family health concerns** do not show a significant direct effect.

**Equation 2: Perceived Harm to Community (`cc4_comm_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.485** | *** p < 0.01 |
| `cc4_famheal_lag` | **0.143** | *** p < 0.01 |
| `cc_pol_tax_lag` | **0.148** | *** p < 0.01 |

→ Perceptions of community-level harm are highly stable and also shaped by **prior support for climate policy**, suggesting a feedback mechanism where prior endorsement of action may increase perceptions of harm.

**Equation 3: Perceived Harm to Family Health (`cc4_famheal_t`)**  
| Predictor | Coefficient | Significance |
|-----------|-------------|--------------|
| `cc4_comm_lag` | **0.303** | *** p < 0.01 |
| `cc4_famheal_lag` | **0.337** | *** p < 0.01 |
| `cc_pol_tax_lag` | **0.124** | *** p < 0.01 |

→ Perceived harm to family health is also self-reinforcing and shaped by both other variables. Prior perceptions of harm to the community have a strong influence on subsequent perceptions of personal harm, as does support for climate policy.

### Interpretation
This Panel VAR model shows that both **community-level and personal-level climate harm perceptions** are dynamically linked with **policy support**. Community-level harm perception appears particularly influential in predicting **future support for carbon taxes**. At the same time, **policy support also contributes to increased harm perception**, suggesting a reciprocal dynamic.

These findings are consistent with theoretical models of climate opinion formation, where perceptions and policy preferences evolve together. Notably, while both forms of perceived harm are internally stable, **community-level concern appears more strongly connected to policy attitudes** than personal-level concern.

---

We will now estimate the exact same **Panel VAR(1)** model with `cc_pol_car` (support for emissions standards) as the policy support variable, replacing `cc_pol_tax`. This allows us to directly compare whether perceived harm affects **support for regulatory solutions** (like car standards) differently from **market-based solutions** (like carbon taxes).

### Variables for the Updated System
| Variable      | Description                                   |
|---------------|-----------------------------------------------|
| `cc_pol_car`  | Support for emissions standards (new outcome) |
| `cc4_comm`    | Perceived harm to local community             |
| `cc4_famheal` | Perceived harm to family health               |

We will create lagged versions of all three and estimate the same three equations system.

In [6]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Sort and create lagged variables
data = data.sort_values(by=['PID', 'wave'])
data['cc4_comm_lag'] = data.groupby('PID')['cc4_comm'].shift(1)
data['cc4_famheal_lag'] = data.groupby('PID')['cc4_famheal'].shift(1)
data['cc_pol_car_lag'] = data.groupby('PID')['cc_pol_car'].shift(1)

# Drop missing rows
panel_var_data = data.dropna(subset=['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag'])

# Equation 1: cc_pol_car_t
X1 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y1 = panel_var_data['cc_pol_car']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 2: cc4_comm_t
X2 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y2 = panel_var_data['cc4_comm']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Equation 3: cc4_famheal_t
X3 = panel_var_data[['cc4_comm_lag', 'cc4_famheal_lag', 'cc_pol_car_lag']]
y3 = panel_var_data['cc4_famheal']
X3 = sm.add_constant(X3)
model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Display results
print(summary_col([model1, model2, model3],
                  stars=True,
                  model_names=['cc_pol_car_t', 'cc4_comm_t', 'cc4_famheal_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


                cc_pol_car_t cc4_comm_t cc4_famheal_t
-----------------------------------------------------
const           0.9345***    0.5058***  0.4855***    
                (0.0857)     (0.0696)   (0.0698)     
cc4_comm_lag    0.1431***    0.4792***  0.3195***    
                (0.0294)     (0.0327)   (0.0341)     
cc4_famheal_lag 0.0196       0.1466***  0.3421***    
                (0.0263)     (0.0292)   (0.0349)     
cc_pol_car_lag  0.6020***    0.1701***  0.0870***    
                (0.0244)     (0.0214)   (0.0217)     
R-squared       0.4613       0.4840     0.4475       
R-squared Adj.  0.4604       0.4831     0.4466       
N               1722         1722       1722         
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Panel VAR Analysis – RQ3 (Emissions Standards as Outcome)
To complement the prior analysis of support for carbon taxes, I estimated a Panel VAR(1) model using `cc_pol_car` (support for stronger emissions standards for car manufacturers) as the primary policy support outcome. The model includes two perceived harm variables, `cc4_comm` (harm to local communities) and `cc4_famheal` (harm to family health), as endogenous variables in the system. Each variable was regressed on its own lag and the lags of the other two. The model was estimated using pooled OLS with clustered standard errors by respondent (PID) and includes 1,722 observations from three waves.

### Results
**Equation 1: Support for Emissions Standards (`cc_pol_car_t`)**  
| Predictor          | Coefficient | Significance    |
|--------------------|-------------|-----------------|
| `cc4_comm_lag`     | **0.143**   | *** p < 0.01    |
| `cc4_famheal_lag`  | 0.020       | Not significant |
| `cc_pol_car_lag`   | **0.602**   | *** p < 0.01    |
| R²                 | 0.461       | —               |

→ Support for emissions standards is highly stable over time and significantly influenced by previous perceptions of harm to the local community. Perceived harm to family health does not show a significant direct effect on this form of policy support.

**Equation 2: Perceived Harm to Local Community (`cc4_comm_t`)**  
| Predictor          | Coefficient | Significance |
|--------------------|-------------|--------------|
| `cc4_comm_lag`     | **0.479**   | *** p < 0.01 |
| `cc4_famheal_lag`  | **0.147**   | *** p < 0.01 |
| `cc_pol_car_lag`   | **0.170**   | *** p < 0.01 |
| R²                 | 0.484       | —            |

→ Community-level harm perception is highly stable and positively influenced by previous policy support and perceptions of personal harm.

**Equation 3: Perceived Harm to Family Health (`cc4_famheal_t`)**  
| Predictor          | Coefficient | Significance |
|--------------------|-------------|--------------|
| `cc4_comm_lag`     | **0.320**   | *** p < 0.01 |
| `cc4_famheal_lag`  | **0.342**   | *** p < 0.01 |
| `cc_pol_car_lag`   | **0.087**   | *** p < 0.01 |
| R²                 | 0.448       | —            |

→ Perceptions of harm to family health are influenced by both their own lagged values and by prior community-level harm perceptions. Support for emissions standards also has a modest positive effect on subsequent personal harm perceptions.

### Interpretation
Support for emissions standards is not only self-reinforcing but also significantly predicted by prior perceptions of harm to local communities. These findings mirror the patterns observed in the carbon tax model, suggesting that **community-level concern** is a more robust and consistent predictor of policy support than **personal-level concern**. Moreover, the feedback effects from policy support to harm perceptions highlight the potential role of motivated reasoning or belief updating over time.

---

### RQ4. Does political affiliation or ideology moderate the relationship between perceptions of harm and willingness to pay?
> **RQ4:** Does political affiliation or ideology **moderate** the relationship between perceptions of harm and willingness to pay?

This is a **moderation question**, asking:
- Is the effect of **harm perception** on **WTP** different depending on **political identity**?

#### Moderation ≠ What Panel VAR Does
Panel VAR estimates **dynamic feedback** between variables over time, **treating everything as endogenous**. It doesn’t easily accommodate:
- **Time-invariant moderators** (like `pol_ideology`)
- **Interaction terms** (e.g., `cc4_comm × ideology`)

### What We *Can* Do with Panel VAR for RQ4
We *can* estimate a Panel VAR system with:
- `ccSolve` (willingness to pay)
- `cc4_comm` (harm perception)

and **run it separately by ideological group** (e.g., liberals vs conservatives).

This gives us a **subgrouped Panel VAR**, which lets us explore:

> "Is the dynamic link between harm perception and WTP different across ideological groups?"

### How to Do It: Subgrouped Panel VAR
#### 1. Create ideology groups and run separate Panel VARs for each group
#### 2. Compare the coefficients for `cc4_comm_lag` in each group
- If it's **larger among conservatives**, harm perception influences WTP more strongly there.
- If it's **larger among liberals**, it matters more for them.

In [8]:
data['ideology_group'] = data['pol_ideology'].apply(
    lambda x: 'liberal' if x >= 4 else ('conservative' if x <= 2 else 'moderate')
)

for group in ['conservative', 'liberal']:
    subgroup = data[data['ideology_group'] == group].copy()
    subgroup = subgroup.sort_values(by=['PID', 'wave'])

    # Create lags
    subgroup['ccSolve_lag'] = subgroup.groupby('PID')['ccSolve'].shift(1)
    subgroup['cc4_comm_lag'] = subgroup.groupby('PID')['cc4_comm'].shift(1)
    subgroup = subgroup.dropna(subset=['ccSolve_lag', 'cc4_comm_lag'])

    # Model 1: WTP as outcome
    X1 = subgroup[['ccSolve_lag', 'cc4_comm_lag']]
    y1 = subgroup['ccSolve']
    X1 = sm.add_constant(X1)
    model = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': subgroup['PID']})

    print(f"\nSubgroup: {group.upper()}")
    print(model.summary())


Subgroup: CONSERVATIVE
                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     12.20
Date:                Sun, 13 Apr 2025   Prob (F-statistic):           8.39e-06
Time:                        18:33:25   Log-Likelihood:                -606.18
No. Observations:                 516   AIC:                             1218.
Df Residuals:                     513   BIC:                             1231.
Df Model:                           2                                         
Covariance Type:              cluster                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.1229 

### Panel VAR Analysis – RQ4 (Subgrouped by Political Ideology)
To assess whether political ideology moderates the relationship between climate harm perception and willingness to pay (WTP) for climate solutions, I estimated separate Panel VAR(1) models for conservative and liberal participants. The key dependent variable was `ccSolve` (willingness to pay), with lagged predictors including `ccSolve_lag` and `cc4_comm_lag` (perceived harm to local communities).

Models were estimated using pooled OLS with standard errors clustered at the individual level (`PID`).

### Results
#### Conservative Participants (pol_ideology = 1–2)
| Predictor       | Coefficient | Significance    |
|-----------------|-------------|-----------------|
| `cc4_comm_lag`  | **0.129**   | *** p < 0.01    |
| `ccSolve_lag`   | 0.085       | Not significant |
| R-squared       | 0.052       | —               |

→ Among conservatives, **perceived harm to the local community at t–1 significantly increases willingness to pay at time t**. This suggests that harm perception is a strong predictor of climate-related financial support in this subgroup.

#### Liberal Participants (pol_ideology = 4–5)

| Predictor       | Coefficient | Significance |
|-----------------|-------------|--------------|
| `cc4_comm_lag`  | 0.003       | Not significant |
| `ccSolve_lag`   | **0.154**   | ** p < 0.01 |
| R-squared       | 0.022       | —            |

→ Among liberals, **willingness to pay is primarily predicted by its own past value**, with **no significant contribution from perceived harm**. This suggests that liberals’ willingness to pay is more stable and less influenced by short-term shifts in perceived community-level harm.

### Interpretation
These findings provide evidence that **political ideology moderates the relationship between harm perception and willingness to pay**. Among conservatives, harm perception plays a **significant causal role** in increasing willingness to pay for climate solutions, whereas among liberals, WTP appears **less sensitive to harm perception** and more **internally consistent over time**.

This is consistent with theoretical expectations: liberals may hold a more **ideologically grounded** climate position, whereas conservative support may depend more on **issue salience** or **personal/community relevance**.